In [7]:
pip install --upgrade pip


     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.1 MB 640.0 kB/s eta 0:00:04
     ---------------------------------------- 0.0/2.1 MB 640.0 kB/s eta 0:00:04
      --------------------------------------- 0.0/2.1 MB 217.9 kB/s eta 0:00:10
      --------------------------------------- 0.0/2.1 MB 217.9 kB/s eta 0:00:10
      --------------------------------------- 0.0/2.1 MB 217.9 kB/s eta 0:00:10
      --------------------------------------- 0.0/2.1 MB 109.3 kB/s eta 0:00:19
      --------------------------------------- 0.1/2.1 MB 131.3 kB/s eta 0:00:16
     - -------------------------------------- 0.1/2.1 MB 142.6 kB/s eta 0:00:15
     - -------------------------------------- 0.1/2.1 MB 140.6 kB/s eta 0:00:15
     - -------------------------------------- 0.1/2.1 MB 174.7 kB/s eta 0:00:12
     -- ------------------------------------- 0.1/2.1 MB 205.9 kB/s eta 0:00:10
     --- ------------------------------------ 0.2


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\joana\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:
#!pip install transformers
#!pip install gensim
#!pip install spacy #rather use Terminal with:
'''pip install -U spacy
python -m spacy download en_core_web_sm'''
#!pip install nltk

In [19]:
# Step 1 importing packages
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from transformers import BertTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [28]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v2.csv'
df = pd.read_csv(url)
df.head()


,data,player,language
0,{'content': 'Manchester\xa0United\xa0imploded ...,Mitchel Bakker,en
1,{'content': 'Slick Sevilla punish slapdash Man...,Mitchel Bakker,en
2,{'content': 'Manchester United imploded in Sev...,Mitchel Bakker,en
3,{'content': 'Bayer Leverkusen has reached the ...,Mitchel Bakker,en
4,{'content': 'Leverkusen\'s players celebrate a...,Mitchel Bakker,en


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   data      440 non-null    object
 1   player    440 non-null    object
 2   language  440 non-null    object
dtypes: object(3)
memory usage: 10.4+ KB


In [34]:
df['player'].value_counts()

Moussa Diaby         130
Jeremie Frimpong     111
Jonathan Tah          68
Exequiel Palacios     52
Mitchel Bakker        42
Mykhaylo Mudryk       30
Piero Hincapie         7
Name: player, dtype: int64

In [33]:
df['language'].value_counts()
#for more about that, pls see "Language_check.ipynb"

de    309
en     99
es     32
Name: language, dtype: int64

In [41]:
data_lower = df.copy()

In [42]:
data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

,data,player,language
0,{'content': 'manchester\xa0united\xa0imploded ...,mitchel bakker,en
1,{'content': 'slick sevilla punish slapdash man...,mitchel bakker,en
2,{'content': 'manchester united imploded in sev...,mitchel bakker,en
3,{'content': 'bayer leverkusen has reached the ...,mitchel bakker,en
4,{'content': 'leverkusen\'s players celebrate a...,mitchel bakker,en


In [43]:
print(data_lower.iloc[0, 0])
'''
To Dos dervied from that:
- cut in the beginning: {'content': '
- cut in the end: \nAdvertisement\nAdvertisement\nAdvertisement'}
'''

{'content': 'manchester\xa0united\xa0imploded in seville in the europa league on thursday while juventus completed a good evening for the club by holding on away to sporting and advancing to the semi-finals.in the quarter-final second legs,\xa0united\xa0succumbed 3-0 at sevilla to go out 5-2.juventus hung on to draw 1-1 with sporting in lisbon to advance 2-1 on aggregate while bayer leverkusen won 4-1 in belgium to eliminate union saint-gilloise 5-2.jose mourinho\'s roma beat feyenoord 4-1 after extra time to advance 4-2.united, who led by two goals with six minutes left in the first leg before conceding a pair of late own goals, completed the collapse in spain.sponsored by vuuklegoalkeeper david de gea had a disastrous night as lacklustre\xa0united\xa0were dispatched by the six-time competition winners.after eight minutes de gea rolled a pass to an isolated harry maguire, scorer of one of the first-leg own goals. this time, ambushed by three opponents, he gave the ball away and yousse

"\nTo Dos dervied from that:\n- cut in the beginning: {'content': '\n- cut in the end: \nAdvertisement\nAdvertisement\nAdvertisement'}\n"

In [47]:
pattern_beg = r"^{\'content\': \'\\n"
pattern_end = r"\\nadvertisement\\nadvertisement\\nadvertisement'\}$"

data_lower['data'] = data_lower['data'].str.replace(pattern_beg, '', flags=re.MULTILINE)
data_lower['data'] = data_lower['data'].str.replace(pattern_end, '', flags=re.MULTILINE)
#print(data_lower.iloc[0, 0])
data_lower['data'].head()

C:\Users\joana\AppData\Local\Temp\ipykernel_7856\2207826794.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data_lower['data'] = data_lower['data'].str.replace(pattern_beg, '', flags=re.MULTILINE)
C:\Users\joana\AppData\Local\Temp\ipykernel_7856\2207826794.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data_lower['data'] = data_lower['data'].str.replace(pattern_end, '', flags=re.MULTILINE)


0    {'content': 'manchester\xa0united\xa0imploded ...
1    {'content': 'slick sevilla punish slapdash man...
2    {'content': 'manchester united imploded in sev...
3    {'content': 'bayer leverkusen has reached the ...
4    {'content': 'leverkusen\'s players celebrate a...
Name: data, dtype: object

In [49]:
data_lower['data'] = data_lower['data'].apply(lambda x: re.sub(r"^{\'content\': \'\\n", "", str(x)))
data_lower['data'] = data_lower['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement'\}$", "", str(x)))
print(data_lower.iloc[0, 0])


{'content': 'manchester\xa0united\xa0imploded in seville in the europa league on thursday while juventus completed a good evening for the club by holding on away to sporting and advancing to the semi-finals.in the quarter-final second legs,\xa0united\xa0succumbed 3-0 at sevilla to go out 5-2.juventus hung on to draw 1-1 with sporting in lisbon to advance 2-1 on aggregate while bayer leverkusen won 4-1 in belgium to eliminate union saint-gilloise 5-2.jose mourinho\'s roma beat feyenoord 4-1 after extra time to advance 4-2.united, who led by two goals with six minutes left in the first leg before conceding a pair of late own goals, completed the collapse in spain.sponsored by vuuklegoalkeeper david de gea had a disastrous night as lacklustre\xa0united\xa0were dispatched by the six-time competition winners.after eight minutes de gea rolled a pass to an isolated harry maguire, scorer of one of the first-leg own goals. this time, ambushed by three opponents, he gave the ball away and yousse

In [ ]:
#how can /want we use patterns? Eg for illnesses?